In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import random
import copy
import math
import pandas as pd
import time
DIR = '/content/drive/MyDrive/ToiUuLapLich/data'
DIR_SET = '/content/drive/MyDrive/ToiUuLapLich/set_conflicts'
DIR_RAND = '/content/drive/MyDrive/ToiUuLapLich/rnd_conflicts'
%cd '/content/drive/MyDrive/ToiUuLapLich'


/content/drive/MyDrive/ToiUuLapLich


In [3]:
os.listdir()

['data',
 'Greedy.ipynb',
 'TabuSearch.ipynb',
 'LocalSearch.ipynb',
 'rnd_conflicts',
 'set_conflicts',
 'result',
 'Bản sao của LocalSearch.ipynb',
 'Untitled0.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'AnInterationLocalSearch.ipynb']

## Data Preprocessing

### Read Data

In [4]:
def get_map(lst):
  sorted_lst = sorted(lst, reverse=True)

  index_map = {}
  occurrence_count = {}
  for index, element in enumerate(lst):
      if element in occurrence_count:
          occurrence_count[element] += 1
          index_map[index] = sorted_lst.index(element) + occurrence_count[element] - 1
      else:
          occurrence_count[element] = 1
          index_map[index] = sorted_lst.index(element)
  return index_map, sorted_lst

In [5]:
def read_data(data_file_name):
  with open(data_file_name, "r") as f:
    r = f.readlines()
    number_subject = int(r[0])
    student_per_subject = [int(u) for u in r[1][:-1].split(" ")]
    number_room = int(r[2])
    place_per_room = [int(u) for u in r[3][:-1].split(" ")]
    number_constrain = int(r[4])

    index_map, student_per_subject = get_map(student_per_subject)
    student_per_subject = np.asarray(student_per_subject)

    place_per_room.sort(reverse=True)
    place_per_room = np.asarray(place_per_room)

    if(r[-1][-1] != '\n'):
      r[-1] += "\n"

    constrain = []
    for i in range(number_constrain):
      each_constrain = [int(u) for u in r[5+i][:-1].split(" ")]
      constrain.append(each_constrain)

    matrix_constrain = np.zeros((number_subject, number_subject))
    for iter in constrain:
      i = index_map[iter[0]]
      j = index_map[iter[1]]
      matrix_constrain[i][j] = 1
      matrix_constrain[j][i] = 1
        
    return number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain

# TEST
number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(f'{DIR}/data_10_2_(0).txt')


In [6]:
def greedy_read_data(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain):
  # Solve
  solve = []
  kip = 0
  subject_checked = np.zeros((number_subject,))

  while(np.sum(subject_checked) < number_subject):
    kip += 1
    this_kip = []
    iter_room = 0
    iter_sub = 0
    conflict = np.zeros((number_subject,), dtype=int)
    while((iter_sub < number_subject) and (iter_room < number_room)):
      if (subject_checked[iter_sub] == 1) or (conflict[iter_sub] == 1):  # already checked
        iter_sub += 1
        continue
      else: # unchecked
        if(place_per_room[iter_room] >= student_per_subject[iter_sub]): # fit the room -> checked
          subject_checked[iter_sub] = 1

          # TO-DO ADD CONFLICT
          for i in range(number_subject):
            if matrix_constrain[iter_sub][i] == 1:
              conflict[i] = 1
            
          # ADD CONFLICT
          this_kip.append(iter_sub)
          iter_room += 1
          iter_sub += 1
          
        else:
          iter_sub += 1
    solve.append(this_kip)

  return kip, solve


In [7]:
# number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(data_file)
# kip, solve = greedy_read_data(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain)
# print(kip)
# print(solve)

NameError: ignored

### Evaluate Greedy

In [ ]:
greedy_result = pd.DataFrame(columns={'datafile', 'result', 'time', 'detailResult'})[['datafile', 'result', 'time', 'detailResult']]
greedy_result 

,datafile,result,time,detailResult


In [ ]:
%cd $DIR_SET
greedy_result = pd.DataFrame(columns={'datafile', 'result', 'time', 'detailResult'})[['datafile', 'result', 'time', 'detailResult']]
for data_file in os.listdir():
  if 'txt' in data_file:
    
    file_name = os.path.join(DIR_SET, data_file)
    start_time = time.time()

    number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(file_name)
    kip, solve = greedy_read_data(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain)  

    runtime = time.time() - start_time

    greedy_result.loc[len(greedy_result.index)] = [data_file, kip, runtime, solve]

greedy_result = greedy_result.sort_values(by=['datafile'])
greedy_result

/content/drive/MyDrive/ToiUuLapLich/set_conflicts


,datafile,result,time,detailResult
74,data_10_2_12_(0).txt,9,0.002082,"[[0], [1, 9], [2], [3], [4], [5], [6], [7], [8]]"
13,data_10_2_12_(1).txt,6,0.003310,"[[0, 1], [2, 3], [4, 5], [6], [7, 8], [9]]"
33,data_10_2_12_(2).txt,8,0.001900,"[[0, 8], [1, 9], [2], [3], [4], [5], [6], [7]]"
58,data_10_2_12_(3).txt,5,0.001592,"[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]"
94,data_10_2_12_(4).txt,6,0.002591,"[[0, 1], [2, 3], [4, 5], [6, 8], [7], [9]]"
...,...,...,...,...
43,data_80_20_80_(0).txt,6,0.005449,"[[0, 1, 2, 3, 4, 5, 6, 7, 16, 22, 30, 43, 57, ..."
137,data_80_20_80_(1).txt,6,0.004973,"[[0, 1, 2, 3, 4, 5, 24, 25, 29, 43, 44, 45, 46..."
40,data_80_20_80_(2).txt,5,0.005022,"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 38, 39, 41, 42, 4..."
20,data_80_20_80_(3).txt,5,0.005322,"[[0, 1, 2, 3, 7, 13, 19, 20, 26, 32, 33, 37, 3..."


In [ ]:
greedy_result.to_csv('/content/drive/MyDrive/ToiUuLapLich/result/set_greedy.csv')

# Local Search

## Algorithm

### Some Function


In [9]:
def max_match(list_subject, student_per_subject, number_room, place_per_room):
  sub_index = 0
  room_index = 0
  sub_len = len(list_subject)
  room_len = number_room
  accept_sub = []
  reject_sub = []

  while((sub_index < sub_len) and (room_index < room_len)):
    if(student_per_subject[list_subject[sub_index]] <= place_per_room[room_index]):
      accept_sub.append(list_subject[sub_index])
      sub_index += 1
      room_index += 1
    else:
      reject_sub.append(list_subject[sub_index])
      sub_index += 1

  while(sub_index < sub_len):
    reject_sub.append(list_subject[sub_index])
    sub_index += 1

  return accept_sub, reject_sub

In [10]:
def initial_solution_greedy(k, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain):
  solution = []
  subject_checked = np.zeros((number_subject,))

  for i in range(k):
    conflict = np.zeros((number_subject,), dtype=int)
    kip_sol = []
    room_index = 0
    for sub_index in range(number_subject):
      if((subject_checked[sub_index] == 1) or (conflict[sub_index] == 1)):
        continue
      else:
        if(student_per_subject[sub_index] <= place_per_room[room_index]):
          # ADD THIS SUBJECT
          kip_sol.append(sub_index)

          # CHECKED
          subject_checked[sub_index] = 1

          # ADD CONFLICT
          for i in range(number_subject):
            if matrix_constrain[sub_index][i] == 1:
              conflict[i] = 1

          # INCREASE
          room_index += 1
          if(room_index >= number_room): break

    # ADD KIP's SOLUTION
    solution.append(kip_sol)

  unassign = []
  for i in range(number_subject):
    if subject_checked[i] == 0:
      unassign.append(i)

  return solution, unassign

In [11]:
def initial_solution_maxmatch(k, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain):
  solution = []
  subject_checked = np.zeros((number_subject,))

  for i in range(k):
    conflict = np.zeros((number_subject,), dtype=int)
    kip_sol = []
    for sub_index in range(number_subject):
      if((subject_checked[sub_index] == 1) or (conflict[sub_index] == 1)):
        continue
      else:
        # ADD THIS SUBJECT
        kip_sol.append(sub_index)

        # ADD CONFLICT
        for i in range(number_subject):
            if matrix_constrain[sub_index][i] == 1:
              conflict[i] = 1

    acc, rej = max_match(kip_sol, student_per_subject, number_room, place_per_room)
    for sub in acc:
      subject_checked[sub] = 1
    # ADD KIP's SOLUTION
    solution.append(acc)

  unassign = []
  for i in range(number_subject):
    if subject_checked[i] == 0:
      unassign.append(i)

  return solution, unassign

In [12]:
def generate_conflict(sub_list, matrix_constrain):
   conflict = np.zeros((len(matrix_constrain),), dtype=int)
   for sub_index in sub_list:
     for i in range(len(matrix_constrain)):
        if matrix_constrain[sub_index][i] == 1:
          conflict[i] = 1
   return conflict

In [13]:
def delta_st(list1, list2, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain):
  student_list1 = 0
  student_list2 = 0
  degree_list1 = 0
  degree_list2 = 0

  for i in range(len(list1)):
    student_list1 += student_per_subject[list1[i]]
    student_list2 += student_per_subject[list2[i]]

    degree_list1 += np.sum(matrix_constrain[list1[i]])
    degree_list2 += np.sum(matrix_constrain[list2[i]])

  return student_list1 - student_list2 + degree_list1 - degree_list2



In [14]:
def naive_move_local_search(t1, t2, r1, sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain):
  # Keep copy
  sol_copy = copy.deepcopy(sol)
  unassign_copy = copy.deepcopy(unassign)
  
  # RANDOM SELECT ELEMENT
  r2 = random.randint(0, len(sol[r1])-1)
  remove_sub = sol[r1][r2]

  # REMOVE RANDOM ELEMENT
  sol[r1].remove(remove_sub)
  conflict = generate_conflict(sol[r1], matrix_constrain) # [0,0,1,0,1,1,0,1]

  # BUILD NEW LIST
  for sub_index in unassign:
    if conflict[sub_index] == 1:
      continue
    else:
      sol[r1].append(sub_index)
      unassign.remove(sub_index)
      
      for i in range(number_subject):
        if matrix_constrain[sub_index][i] == 1:
          conflict[i] = 1
  
  # USING MAX_MATCH ON NEW LIST
  accept_sub, reject_sub = max_match(sol[r1], student_per_subject, number_room, place_per_room)

  # UPDATE AFTER USING MAX_MATCH 
  sol[r1] = accept_sub
  unassign = unassign + reject_sub
  unassign.append(remove_sub)

  # Accept the new solution?
  delta = len(sol[r1]) - len(sol_copy[r1])
  if delta == 0:
    this_delta_st = delta_st(sol[r1], sol_copy[r1], number_subject, student_per_subject, number_room, place_per_room, matrix_constrain)
  r = random.uniform(0,1)
  if (delta > 0):# or ((delta<0) and (r < math.exp(delta/t1))): # or ((delta==0) and (this_delta_st>=0)) or ((delta==0) and (this_delta_st<0) and (r < math.exp(this_delta_st/t2))):
    return sol, unassign
  else:
    return sol_copy, unassign_copy


In [15]:
r = random.uniform(0,1)
r

0.48231779351742665

### Improvement_perturbation procedure

In [16]:
def swap_period_move(sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain, checkpoint = 0):
  
  clone_sol = copy.deepcopy(sol)
  clone_unassign = copy.deepcopy(unassign)

  len_sol = len(sol)
  # RANDOM SELECT 2 PERIODS
  p1 = random.randint(0, len_sol-1)
  p2 = p1
  while(p2 == p1):
    p2 = random.randint(0, len_sol-1)
  
  r = random.randint(0, len(sol[p1])-1)
  remove = sol[p1][r]

  #conflict
  conflict_with_remove = matrix_constrain[remove]
  sol[p1].remove(remove)

  remove_set_2 = [] # set in periods 2 conflict with remove element
  for sub in sol[p2]:
    if conflict_with_remove[sub] == 1:
      # sol[p2].remove(sub)
      remove_set_2.append(sub)
  for sub in remove_set_2:
    sol[p2].remove(sub)

  sol[p2].append(remove)
  conflict_period_1 = generate_conflict(sol[p1], matrix_constrain) # conflict after remove element

  # if remove in conflict break --> return clone
  for sub in remove_set_2:
    if conflict_period_1[sub] == 1:
      return clone_sol, clone_unassign, checkpoint

  sol[p1] = sol[p1] + remove_set_2

  acc1, rej1 = max_match(sol[p1], student_per_subject, number_room, place_per_room)
  acc2, rej2 = max_match(sol[p2], student_per_subject, number_room, place_per_room)
  acc1_cp = copy.deepcopy(acc1)
  acc2_cp = copy.deepcopy(acc2)

  if ((len(rej1) > 0) or (len(rej2) > 0)):
    return clone_sol, clone_unassign, checkpoint

  conflict_set1 = generate_conflict(sol[p1], matrix_constrain)
  conflict_set2 = generate_conflict(sol[p2], matrix_constrain)

  unassign_remove = []
  for sub in unassign:
    if conflict_set1[sub] == 0:
      unassign_remove.append(sub)
      sol[p1].append(sub)
      for i in range(number_subject):
        if matrix_constrain[sub][i] == 1:
          conflict_set1[i] = 1

    elif conflict_set2[sub] == 0:
      unassign_remove.append(sub)
      sol[p2].append(sub)
      for i in range(number_subject):
        if matrix_constrain[sub][i] == 1:
          conflict_set2[i] = 1

  for sub in unassign_remove:
    unassign.remove(sub)

  acc1, rej1 = max_match(sol[p1], student_per_subject, number_room, place_per_room)
  acc2, rej2 = max_match(sol[p2], student_per_subject, number_room, place_per_room)

  sol[p1] = acc1
  sol[p2] = acc2

  unassign = unassign + rej1 + rej2
  unassign = np.unique(unassign).tolist()

  improve = len(clone_unassign) - len(unassign)
  if(improve > checkpoint):
    return sol, unassign, improve

  return clone_sol, clone_unassign, checkpoint

### SA Based Local Search

In [17]:
def SA_base_local_search(t1, t2, sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain):

  each_temp = 5
  l = 15
  

  while((each_temp > 0) and (len(unassign)>0)):
    for i in range(l):
      # Generate a random permutation of solution
      len_sol = len(sol)

      # Iteration len periods
      for i in range(len_sol):
        sol, unassign = naive_move_local_search(t1, t2, i, sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain)
    
    t1 = 1/(1/t1 + 0.2)
    t2 = 1/(1/t2 + 0.09)
      
    each_temp -= 1

  return sol, unassign, t1, t2                    

### k-color procedure

In [18]:
def k_color(k, number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain):
  #Init solution
  sol, unassign = initial_solution_greedy(k, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain)
  best_solution = copy.deepcopy(sol)
  best_unassign = copy.deepcopy(unassign)

  #Init parameter
  number_iter = 150
  t1 = 5
  t2 = 30
  m = 10


  while(number_iter > 0):
    #ITERATION
    number_iter -= 1

    # SA based local search
    sol, unassign, t1, t2 = SA_base_local_search(t1, t2, sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain)

    # improvement_pẻturbation
    checkpoint = 0
    for i in range(m):
      sol, unassign, checkpoint  = swap_period_move(sol, unassign, number_subject, student_per_subject, number_room, place_per_room, matrix_constrain, checkpoint)

    # Check best solution
    if(len(unassign) < len(best_unassign)):
      best_solution = copy.deepcopy(sol)
      best_unassign = copy.deepcopy(unassign)

    if(len(best_unassign) == 0):
      break

  if(len(best_unassign) == 0): return True, best_solution
  else: return False, False


### Optimize Procedure

In [19]:
def optimize_solve(data_file):
  # READ DATA
  number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain = read_data(data_file)

  # GET GREEDY SOLUTION
  len_greedy, sol = greedy_read_data(number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain)
  
  # Init k as greedy solution
  k = len_greedy - 1
  print("len greedy: ", len_greedy)
  final_result = sol

  while(1):
    verify_k_sol, best_solution = k_color(k, number_subject, student_per_subject, number_room, place_per_room, number_constrain, constrain, matrix_constrain)
    print(f"k = {k}, result = {verify_k_sol}")
    if(verify_k_sol):
      final_result = best_solution
      k -= 1
    else:
      return k + 1, final_result
      break

  return -1


## SOLUTION

In [ ]:
data_file = f'{DIR_SET}/data_20_4_60_(4).txt'
len_sol, final_solution = optimize_solve(data_file)
print(len_sol)
print("final solution:", final_solution)

len greedy:  10
k = 9, result = False
False
10
final solution: [[0, 10, 13, 18], [1, 11, 14, 19], [2, 12, 15], [3, 16], [4, 17], [5], [6], [7], [8], [9]]


In [23]:
data_file = f'data_20_4_60_(4).txt'


20

## Evaluate Local Search

In [24]:
%cd $DIR_SET
SA_result = pd.DataFrame(columns={'datafile', 'result', 'time', 'detailResult'})[['datafile', 'result', 'time', 'detailResult']]
for data_file in os.listdir():
  if ('txt' in data_file) and (int(data_file.split("_")[1]) <=50):
    
    file_name = os.path.join(DIR_SET, data_file)
    start_time = time.time()

    len_sol, final_solution = optimize_solve(data_file)

    runtime = time.time() - start_time

    SA_result.loc[len(SA_result.index)] = [data_file, len_sol, runtime, final_solution]

SA_result = SA_result.sort_values(by=['datafile'])
SA_result

/content/drive/MyDrive/ToiUuLapLich/set_conflicts
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  11
k = 10, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  12
k = 11, result = False
len greedy:  7
k = 6, result = False
len greedy:  6
k = 5, result = False
len greedy:  5
k = 4, result = False
len greedy:  6
k = 5, result = False
len greedy:  8
k = 7, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  7
k = 6, result = False
len greedy:  7
k = 6, result = False
len greedy:  8
k = 7, result = False
len greedy:  10
k = 9, result = False
len greedy:  7
k = 6, result = False
len greedy:  8
k = 7, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  6
k = 5, result = False
len greedy:  5
k = 4

,datafile,result,time,detailResult
48,data_10_2_12_(0).txt,9,6.022459,"[[0], [1, 9], [2], [3], [4], [5], [6], [7], [8]]"
6,data_10_2_12_(1).txt,6,3.396116,"[[0, 1], [2, 3], [4, 5], [6], [7, 8], [9]]"
18,data_10_2_12_(2).txt,8,5.683700,"[[0, 8], [1, 9], [2], [3], [4], [5], [6], [7]]"
36,data_10_2_12_(3).txt,5,2.647652,"[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]"
67,data_10_2_12_(4).txt,6,3.448851,"[[0, 1], [2, 3], [4, 5], [6, 8], [7], [9]]"
...,...,...,...,...
60,data_50_10_80_(0).txt,7,18.502484,"[[0, 1, 4, 7, 12, 15, 29, 33, 38, 39], [2, 3, ..."
37,data_50_10_80_(1).txt,6,18.654453,"[[0, 1, 2, 3, 4, 21, 23, 27, 46], [5, 6, 7, 8,..."
87,data_50_10_80_(2).txt,7,18.638613,"[[0, 1, 2, 4, 5, 22, 23, 24, 31, 39], [3, 6, 7..."
50,data_50_10_80_(3).txt,7,16.783733,"[[0, 1, 2, 3, 10, 11, 12, 35, 44, 46], [4, 5, ..."


In [27]:
SA_result.to_csv('/content/drive/MyDrive/ToiUuLapLich/result/set_TLS_new.csv')

In [25]:
SA_result

,datafile,result,time,detailResult
48,data_10_2_12_(0).txt,9,6.022459,"[[0], [1, 9], [2], [3], [4], [5], [6], [7], [8]]"
6,data_10_2_12_(1).txt,6,3.396116,"[[0, 1], [2, 3], [4, 5], [6], [7, 8], [9]]"
18,data_10_2_12_(2).txt,8,5.683700,"[[0, 8], [1, 9], [2], [3], [4], [5], [6], [7]]"
36,data_10_2_12_(3).txt,5,2.647652,"[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]"
67,data_10_2_12_(4).txt,6,3.448851,"[[0, 1], [2, 3], [4, 5], [6, 8], [7], [9]]"
...,...,...,...,...
60,data_50_10_80_(0).txt,7,18.502484,"[[0, 1, 4, 7, 12, 15, 29, 33, 38, 39], [2, 3, ..."
37,data_50_10_80_(1).txt,6,18.654453,"[[0, 1, 2, 3, 4, 21, 23, 27, 46], [5, 6, 7, 8,..."
87,data_50_10_80_(2).txt,7,18.638613,"[[0, 1, 2, 4, 5, 22, 23, 24, 31, 39], [3, 6, 7..."
50,data_50_10_80_(3).txt,7,16.783733,"[[0, 1, 2, 3, 10, 11, 12, 35, 44, 46], [4, 5, ..."


In [ ]:
greedy_result

,datafile,result,time,detailResult
74,data_10_2_12_(0).txt,9,0.002258,"[[0], [1, 9], [2], [3], [4], [5], [6], [7], [8]]"
13,data_10_2_12_(1).txt,6,0.002013,"[[0, 1], [2, 3], [4, 5], [6], [7], [8, 9]]"
33,data_10_2_12_(2).txt,8,0.002012,"[[0, 8], [1, 9], [2], [3], [4], [5], [6], [7]]"
58,data_10_2_12_(3).txt,5,0.001764,"[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]"
94,data_10_2_12_(4).txt,6,0.002092,"[[0, 1], [2, 4], [3], [5, 6], [7, 8], [9]]"
...,...,...,...,...
43,data_80_20_80_(0).txt,6,0.013864,"[[0, 1, 2, 3, 4, 5, 6, 7, 16, 22, 30, 43, 57, ..."
137,data_80_20_80_(1).txt,6,0.005007,"[[0, 1, 2, 3, 4, 5, 24, 25, 29, 43, 44, 45, 46..."
40,data_80_20_80_(2).txt,6,0.005015,"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 38, 39, 41, 42, 4..."
20,data_80_20_80_(3).txt,7,0.005986,"[[0, 1, 2, 3, 6, 13, 19, 20, 26, 32, 33, 37, 3..."


In [ ]:
count = 0
for i in range(len(SA_result)):
  this_sa = SA_result.loc[i]
  datafile = this_sa['datafile']
  this_greedy = greedy_result[greedy_result['datafile'] == datafile]
  better = this_sa['result'] - this_greedy['result'].values
  if better < 0:
    count += 1

count

10